<a href="https://colab.research.google.com/github/vigyat1/portfolio/blob/main/exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mediapipe opencv-python numpy scipy


In [ ]:
%%writefile pose_detector.py
import mediapipe as mp
import cv2
import numpy as np


class PoseDetector:
    def __init__(self, static_image_mode=False, smooth=True):
        self.mp_pose = mp.solutions.pose
        self.pose = self.mp_pose.Pose(
            static_image_mode=static_image_mode,
            model_complexity=1,
            smooth_landmarks=smooth,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5,
        )
        self.mp_draw = mp.solutions.drawing_utils

    def detect_pose(self, frame):
        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = self.pose.process(rgb)
        return results

    def extract_keypoints(self, results, width, height):
        if not results.pose_landmarks:
            return None

        points = []
        for lm in results.pose_landmarks.landmark:
            points.append([int(lm.x * width), int(lm.y * height), lm.visibility])
        return np.array(points)

    def draw_landmarks(self, frame, results):
        if results.pose_landmarks:
            self.mp_draw.draw_landmarks(
                frame,
                results.pose_landmarks,
                self.mp_pose.POSE_CONNECTIONS
            )
        return frame


Overwriting pose_detector.py


In [ ]:
%%writefile angle_utils.py
import numpy as np
from scipy.signal import savgol_filter


def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b

    cos_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    angle = np.degrees(np.arccos(np.clip(cos_angle, -1.0, 1.0)))
    return angle


def smooth_signal(values, window=7, poly=3):
    if len(values) < window:
        return values
    return savgol_filter(values, window, poly)


Overwriting angle_utils.py


In [ ]:
%%writefile rules.py
from angle_utils import calculate_angle
import numpy as np


class FormRules:
    def __init__(self):
        self.feedback = {"elbow": "", "wrist": "", "back": ""}

    def check_elbow_angle(self, keypoints, side="left"):
        SHOULDER = 11 if side == "left" else 12
        ELBOW = 13 if side == "left" else 14
        WRIST = 15 if side == "left" else 16

        s = keypoints[SHOULDER][:2]
        e = keypoints[ELBOW][:2]
        w = keypoints[WRIST][:2]

        angle = calculate_angle(s, e, w)

        if angle > 150:
            self.feedback["elbow"] = "Arm extended (down phase)"
        elif 40 <= angle <= 70:
            self.feedback["elbow"] = "Good curl"
        else:
            self.feedback["elbow"] = "Incorrect elbow angle"

        return angle

    def check_wrist_alignment(self, keypoints, side="left"):
        SHOULDER = 11 if side == "left" else 12
        WRIST = 15 if side == "left" else 16

        sh = keypoints[SHOULDER][:2]
        wr = keypoints[WRIST][:2]

        diff = abs(sh[1] - wr[1])

        if diff < 40:
            self.feedback["wrist"] = "Good wrist alignment"
        else:
            self.feedback["wrist"] = "Adjust wrist height"

        return diff

    def check_back_angle(self, keypoints):
        left_sh = keypoints[11][:2]
        left_hip = keypoints[23][:2]

        dy = abs(left_sh[1] - left_hip[1])
        dx = abs(left_sh[0] - left_hip[0])
        angle = np.degrees(np.arctan2(dx, dy))

        if angle < 15:
            self.feedback["back"] = "Back straight"
        else:
            self.feedback["back"] = "Torso leaning"

        return angle


Overwriting rules.py


In [ ]:
%%writefile video_processor.py
import cv2
from pose_detector import PoseDetector
from rules import FormRules
from angle_utils import smooth_signal


class VideoProcessor:
    def __init__(self, input_path, output_path):
        self.detector = PoseDetector()
        self.rules = FormRules()
        self.input_path = input_path
        self.output_path = output_path
        self.elbow_angles = []

    def process_video(self):
        cap = cv2.VideoCapture(self.input_path)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))

        writer = cv2.VideoWriter(
            self.output_path,
            cv2.VideoWriter_fourcc(*"mp4v"),
            fps,
            (width, height)
        )

        while True:
            success, frame = cap.read()
            if not success:
                break

            results = self.detector.detect_pose(frame)
            keypoints = self.detector.extract_keypoints(results, width, height)

            if keypoints is not None:
                elbow_angle = self.rules.check_elbow_angle(keypoints)
                self.rules.check_wrist_alignment(keypoints)
                self.rules.check_back_angle(keypoints)

                frame = self.detector.draw_landmarks(frame, results)

                y = 30
                for rule, msg in self.rules.feedback.items():
                    cv2.putText(frame, f"{rule}: {msg}", (20, y),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                    y += 30

            writer.write(frame)

        cap.release()
        writer.release()
        print("Processing Finished!")


Overwriting video_processor.py


In [ ]:
%%writefile main.py
from video_processor import VideoProcessor

def main():
    input_video = "input_video.mp4"   # file you uploaded
    output_video = "output_with_feedback.mp4"

    processor = VideoProcessor(input_video, output_video)
    processor.process_video()

if __name__ == "__main__":
    main()


Overwriting main.py


In [ ]:
!python main.py


2025-12-12 08:30:52.494017: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765528252.523013    6094 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765528252.538883    6094 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765528252.576482    6094 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765528252.576540    6094 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1765528252.576546    6094 computation_placer.cc:177] computation placer alr

In [ ]:
from google.colab import files
files.download("output_with_feedback.mp4")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>